In [7]:
import numpy as np
import pandas as pd
import re
from textblob import TextBlob
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import pandas as pd
import os

csv_path = os.path.join(os.getcwd(), 'code_gpus', 'twitter-elon-tesla-bert.csv')
df = pd.read_csv(csv_path)
print(df.columns)
selected_columns = df[['Cleaned_Tweets', 'sentiment_score_bert']]

Index(['User', 'Username', 'UTC Date', 'post', 'Binders', 'Permalink',
       'reposts', 'Likes count', 'Impressions', 'Quotes', 'Replies',
       'Bookmarks', 'post value', 'Cleaned_Tweets', 'sentiment_score_bert'],
      dtype='object')
Your burning passion for tesla ….paid off in a way.  It’s a looker indeed, Congrats
